当前主流的大模型都是 decoder-base 的模型结构，分为 prefill 和 decode 两个阶段。其中 decode 时，每次在生成一个 token 时候，都要频繁跟访存交互，加载 KV-Cache，再通过多层网络做完整的前向计算。对于这样的访存密集型的任务，通常会因为访存效率形成训练或推理的瓶颈。

MTP 方法的核心思想是，通过解码阶段的优化，将 1-token 的生成，转变成 multi-token 的生成，从而提升训练和推理的性能。具体来说，在训练阶段，一次生成多个后续 token，可以一次学习多个位置的 label，进而有效提升样本的利用效率，提升训练速度；在推理阶段通过一次生成多个 token，实现成倍的推理加速来提升推理性能。

**Blockwise Parallel Decoding**

Blockwise Parallel Decoding 块并行解码是 google 发表的工作，可以说 MTP 的研究并不是大模型时代的新产物。在学术创新粒度上，其与 Greedy Decoding 是对应的。

如下，主干网络是训练好的多层 decode-only 的 Transformer 网络，经过多层前向计算后，最终隐层输出 $h$ 维度的 $logit$。$logit$ 上面接了多个输出 Head，每个 Head 负责预估一个 token，Head1 负责预估 next token，Head2 负责预估 next next token，以此类推。

<div align="center">
<img src="https://user-images.githubusercontent.com/7529838/48524023-f7e34900-e8c1-11e8-9b61-61202047c263.png" alt="Blockwise Parallel Decoding" width="60%">
</div>

每个 Head 有三层
- 首先是一个所有位置共享的 FFN 层，将 logit 做宽映射，即 $h \rightarrow 4h$；
- 然后过一个 FFN 层，每个位置独立不共享，是特化的 $4h \rightarrow h$；这一层的计算结果再与原始模型的 logit 做残差连接；
- 最后再将结果送入到词表投影层，vocabulary projection 包括一个线性变换和一个 Softmax；Softmax 输出的概率预估了每个词的概率分布，最终通过某种采样方法，如 greedy、beam search 等生成 token；词表投影层同样是各位置共享的，其本身就属于原预训练网络；

每 $k$ 个 token 执行一次上述三阶段过程，相当于两次 forward；对应 $2m/k$ 次前向，可以输出 m 个 token；而以前需要 $m$ 次前向才可以实现 token-by-token 的生成。当 $k = 4$ 时，理论上能节省一半的时间。

<div align="center">
<img src="https://user-images.githubusercontent.com/7529838/48523934-ad61cc80-e8c1-11e8-945c-44a2208a6d9d.png" alt="Blockwise Parallel Decoding" width="60%">
</div>

更进一步，如果将第 $n$ 步的 verify 和第 $n + 1$ 步的 prediction 合并，即每次验证的时候都用若干并行头进行预测（原论文的说法是一个组合的 scoring model and proposal model，即学习到的头和原网络是一体的），则除了最开始第一次需要 $1$ 次 predict 前向，后面的每次 verify 都会同步地完成 predict；一次 verify 得到 $k$ 个 token，想要得到 $m$ 个 token 一共需要 $m/k$ 次前向，加上第一次 predict 共 $m/k + 1$ 次。

**Better & Faster Large Language Models via Multi-token Prediction**

<div align="center">
<img src="https://ar5iv.labs.arxiv.org/html/2404.19737/assets/img/main_fig_col.png" alt="Blockwise Parallel Decoding" width="60%">
</div>

这篇工作的核心思想和大致实现只是上文的改进和现代化。一个共享的 transformer 的主网络，上面接入 4 个并行预估头，针对输入 token 分别预估后续的未来 token。只不过每个头用到了 MHA 和 2 层 FFN，且每个头参数不共享，只有 vocab decoding 的部分参数共享。

图中的误差棒（error bars）并不是标准差或标准误，而是 90% 置信区间（confidence interval, CI）。这里的 bootstrapping（自助采样） 是一种统计方法，核心思想是对原始数据集进行有放回的随机抽样，生成许多“伪数据集”（bootstrap samples），每个伪数据集大小和原始数据集相同；对每个伪数据集重新计算一次指标。

**DeepSeek MTP**
<div align="center">
<img src="https://arxiv.org/html/2412.19437v1/x3.png" alt="Blockwise Parallel Decoding" width="60%">
</div>

文中 densify training signal 的说法很雅致。与 Meta 的工作不同的是，其用独立的 D 个输出头并行预测 D 个附加标记，而 Deepseek 是顺序预测附加标记，并在每个预测深度保持完整的因果链 casual chain。

MTP 策略主要旨在提高主模型的性能，因此在推理过程中，直接丢弃 MTP 模块。Speculative Decoding 是另一套独立的策略。

也就是说，MTP 是从推理阶段的效率角度出发得到的一种解决方案；我们观察到其在形式上与 World Model 的对偶性。因为很自然的，在纯文本模态中的推理，其状态空间和动作空间的关系是
$$s_t = (a_{1}, a_{2}, ..., a_{t-1})$$
假若 MTP 每次多预测的头数是 $k$，则在当前时刻 $t$ 预测之后的 $a_{t}, a_{t+1}, ..., a_{t+k}$，且 $s_t$ 已知，自然也就等同于预测
$$s_{t+i} = s_t \circ (a_{t}, a_{t+1}, ..., a_{t+i}), \quad \text{s.t. } i \in [1, k]$$

**TODO**

- 在 TRL 上实现面向 SFT 和 PPO 以及其他变体的 MTP 后训练；
- 从 World Model 的潜变量动力学（如 Dreamer 等）中提取一种算法设计，应用到 PPOTrainer 过程中；
- 从 MTP 现有研究中提取一些 tricks，应用到 SFT 和 PPO 上，比如最简单的时间窗口大小和频率设置；
- 实验中，希望观察到分数的上升，同时记录 MTP 的状态信息，如实际 tokens 采样、熵分布、权重和 attention map 的 pattern 等等；
- 希望解释出世界模型的实际表现与“预测世界”这一出发点的一致性；给出可解释性；

- 讨论跨模态带来的可能收益，以及背后的哲学；
- 讨论部分，可扩展性讨论 MCTS 在 LLM 和 World Model 之间的关系以及是否衍生对应的 duality；